In [4]:
import json, os
import numpy as np
import pandas as pd
from datetime import datetime
from test.shared import snap, rainbow

import functools

from numba import jit
# detect results for x, y: -2094435, 1681005 took 2.455234 seconds to generate  VERIFIED 2.6 seconds locally
# detect results for x, y: -2033595, 1685955 took 1.879693 seconds to generate
# detect results for x, y: -2098875, 1694895 took 0.986887 seconds to generate  VERIFIED 1.8 seconds locally
pixl_x, pixl_y = -2094435, 1681005
pixl_x, pixl_y = -2098875, 1694895

inputs_url = "https://lcmaphost.com?x={x}&y={y}&acquired=1982-01-01/2015-12-31\
&ubid=LANDSAT_4/TM/SRB1&ubid=LANDSAT_4/TM/SRB2&ubid=LANDSAT_4/TM/SRB3&ubid=LANDSAT_4/TM/SRB4\
&ubid=LANDSAT_4/TM/SRB5&ubid=LANDSAT_4/TM/BTB6&ubid=LANDSAT_4/TM/SRB7&ubid=LANDSAT_4/TM/PIXELQA\
&ubid=LANDSAT_5/TM/SRB1&ubid=LANDSAT_5/TM/SRB2&ubid=LANDSAT_5/TM/SRB3&ubid=LANDSAT_5/TM/SRB4\
&ubid=LANDSAT_5/TM/SRB5&ubid=LANDSAT_5/TM/BTB6&ubid=LANDSAT_5/TM/SRB7&ubid=LANDSAT_5/TM/PIXELQA\
&ubid=LANDSAT_7/ETM/SRB1&ubid=LANDSAT_7/ETM/SRB2&ubid=LANDSAT_7/ETM/SRB3&ubid=LANDSAT_7/ETM/SRB4\
&ubid=LANDSAT_7/ETM/SRB5&ubid=LANDSAT_7/ETM/BTB6&ubid=LANDSAT_7/ETM/SRB7&ubid=LANDSAT_7/ETM/PIXELQA\
&ubid=LANDSAT_8/OLI_TIRS/SRB2&ubid=LANDSAT_8/OLI_TIRS/SRB3&ubid=LANDSAT_8/OLI_TIRS/SRB4&ubid=LANDSAT_8/OLI_TIRS/SRB5\
&ubid=LANDSAT_8/OLI_TIRS/SRB6&ubid=LANDSAT_8/OLI_TIRS/SRB7&ubid=LANDSAT_8/OLI_TIRS/BTB10\
&ubid=LANDSAT_8/OLI_TIRS/PIXELQA".format(x=pixl_x, y=pixl_y)

#dates           = [i.split('=')[1] for i in inputs_url.split('&') if 'acquired=' in i][0]
chips_url       = inputs_url.split('?')[0]
specs_url       = chips_url.replace('/chips', '/chip-specs')
querystr_list   = inputs_url.split('?')[1].split('&')
requested_ubids = tuple([i.replace('ubid=', '') for i in querystr_list if 'ubid=' in i])

# get chip id
chip_x, chip_y = snap(pixl_x, pixl_y)
xindex = int((pixl_x - chip_x) / 30)
yindex = int((chip_y - pixl_y) / 30)
_start = datetime.now()
rbow = rainbow(-2094585, 1682805, specs_url, chips_url, requested_ubids)
print("took: {}".format((datetime.now() - _start).total_seconds()))

#@jit
def dtstr_to_ordinal(dtstr, iso=True):
    """ Return ordinal from string formatted date"""
    _fmt = '%Y-%m-%dT%H:%M:%SZ' if iso else '%Y-%m-%d %H:%M:%S'
    _dt = datetime.strptime(dtstr, _fmt)
    return _dt.toordinal()

took: 27.296124


In [2]:
#%load_ext autoreload
#%autoreload 2
import ccd

In [3]:
row, col = yindex, xindex
rainbow_date_array = np.array(rbow['t'].values)
_start = datetime.now()
result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                     np.array(rbow['blue'].values[:, row, col]),
                     np.array(rbow['green'].values[:, row, col]),
                     np.array(rbow['red'].values[:, row, col]),
                     np.array(rbow['nir'].values[:, row, col]),
                     np.array(rbow['swir1'].values[:, row, col]),
                     np.array(rbow['swir2'].values[:, row, col]),
                     np.array(rbow['thermal'].values[:, row, col]),
                     np.array(rbow['cfmask'].values[:, row, col], dtype=int),
                     params={})
print("took: {}".format((datetime.now() - _start).total_seconds()))

took: 1.68239


/home/caustin/miniconda2/envs/ccd35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [4]:
rainbow_date_array = np.array(rbow['t'].values)
_st = datetime.now()
_results = []
for _x in range(0, 100):
    for _y in range(0, 100):
        row = _y
        col = _x
        _pstart = datetime.now()
        result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                              np.array(rbow['blue'].values[:, row, col]),
                              np.array(rbow['green'].values[:, row, col]),
                              np.array(rbow['red'].values[:, row, col]),
                              np.array(rbow['nir'].values[:, row, col]),
                              np.array(rbow['swir1'].values[:, row, col]),
                              np.array(rbow['swir2'].values[:, row, col]),
                              np.array(rbow['thermal'].values[:, row, col]),
                              np.array(rbow['cfmask'].values[:, row, col], dtype=int),
                              params={})
        if result:
            _results.append((_x, _y))
            print("{}/{} took {} seconds".format(_x, _y, (datetime.now() - _pstart).total_seconds()))

        
print("took: {}".format((datetime.now() - _st).total_seconds()))

0/0 took 1.649852 seconds
0/1 took 1.657786 seconds
0/2 took 1.419819 seconds
0/3 took 1.32656 seconds
0/4 took 1.432151 seconds
0/5 took 1.557231 seconds
0/6 took 1.457413 seconds
0/7 took 1.398303 seconds
0/8 took 1.374576 seconds
0/9 took 1.377771 seconds
0/10 took 1.254215 seconds
0/11 took 1.262047 seconds
0/12 took 1.27317 seconds
0/13 took 1.531479 seconds
0/14 took 1.48254 seconds
0/15 took 1.459803 seconds
0/16 took 1.296999 seconds
0/17 took 1.414475 seconds
0/18 took 1.46508 seconds
0/19 took 1.424325 seconds
0/20 took 1.484795 seconds
0/21 took 1.511357 seconds
0/22 took 1.772455 seconds
0/23 took 1.626123 seconds
0/24 took 1.565168 seconds
0/25 took 1.696821 seconds
0/26 took 1.770563 seconds


KeyboardInterrupt: 

In [4]:
result

{'algorithm': 'lcmap-pyccd:2017.06.20',
 'change_models': [ChangeModel(start_day=724232, end_day=725312, break_day=725392, observation_count=24, change_probability=0, curve_qa=14, blue=SpectralModel(rmse=338.2362517669375, coefficients=array([  5.72314832e-02,  -4.78494556e+01,  -1.35054855e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), intercept=-40542.21402039448, magnitude=0.0), green=SpectralModel(rmse=373.0581077380078, coefficients=array([ -7.71233773e-02,  -9.73256070e+01,  -1.92039993e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), intercept=57243.694048785983, magnitude=0.0), red=SpectralModel(rmse=504.6105508657189, coefficients=array([  3.07450728e-02,  -1.24095968e+02,  -2.86588796e+02,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00]), intercept=-20960.369270100979, magnitude=0.0), nir=SpectralModel(rmse=888.017945834977, coefficients=array([  6.2

In [4]:
15165 / 60 / 60

4.2125

In [4]:
def run_d():
    row, col = yindex, xindex
    rainbow_date_array = np.array(rbow['t'].values)

    result =  ccd.detect([dtstr_to_ordinal(str(pd.to_datetime(i)), False) for i in rainbow_date_array],
                         np.array(rbow['blue'].values[:, row, col]),
                         np.array(rbow['green'].values[:, row, col]),
                         np.array(rbow['red'].values[:, row, col]),
                         np.array(rbow['nir'].values[:, row, col]),
                         np.array(rbow['swir1'].values[:, row, col]),
                         np.array(rbow['swir2'].values[:, row, col]),
                         np.array(rbow['thermal'].values[:, row, col]),
                         np.array(rbow['cfmask'].values[:, row, col], dtype=int),
                         params={})
    if result:
        return True

In [5]:
import profile

In [6]:
profile.run('run_d()', filename="baseline_{}.stats".format(datetime.now().strftime("%m-%d_%H:%M")))

/home/caustin/miniconda2/envs/ccd35/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


'07-18_13:33'

In [5]:
dates = np.array([724232, 724400, 724432, 724448, 724464, 724496, 724576, 724624, 724800, 724848, 724864, 724896])
matrix = np.zeros(shape=(len(dates), 7), order='F')
matrix

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [7]:
dir(matrix)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__r

In [8]:
matrix.ndim

2

In [9]:
type(matrix[0][0])

numpy.float64

In [10]:
avg_days_yr = 365.25
w = 2 * np.pi / avg_days_yr
matrix[:, 0] = dates
matrix[:, 1] = np.cos(w * dates)
matrix[:, 2] = np.sin(w * dates)

In [11]:
matrix

array([[  7.24232000e+05,   5.31316606e-01,  -8.47173337e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.24400000e+05,  -3.03695038e-01,   9.52769292e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.24432000e+05,  -7.57219819e-01,   6.53160122e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.24448000e+05,  -9.06232043e-01,   4.22780658e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.24464000e+05,  -9.87023716e-01,   1.60574545e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.24496000e+05,  -9.25211078e-01,  -3.79452844e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.24576000e+05,   1.93376474e-01,  -9.81124630e-01,
          0.00000000e+00,   0.00

In [15]:
type(matrix[:, 1][0])

numpy.float64

In [13]:
np.cos(w * dates)

array([ 0.53131661, -0.30369504, -0.75721982, -0.90623204, -0.98702372,
       -0.92521108,  0.19337647,  0.85227481, -0.78725998, -0.98702372,
       -0.90623204, -0.55120459])